# Getting Started with Prompt Engineering
by DAIR.AI | Elvis Saravia


This notebook contains examples and exercises to learning about prompt engineering.

We will be using the [OpenAI APIs](https://platform.openai.com/) for all examples. I am using the default settings `temperature=0.7` and `top-p=1`

---

## 1. Prompt Engineering Basics

Objectives
- Load the libraries
- Review the format
- Cover basic prompts
- Review common use cases

Below we are loading the necessary libraries, utilities, and configurations.

In [1]:
%%capture
# update or install the necessary libraries
# !pip install --upgrade openai
!pip install --upgrade langchain
# !pip install --upgrade python-dotenv

In [2]:
import openai
import os
import IPython
from langchain.llms import OpenAI
from dotenv import load_dotenv

Load environment variables. You can use anything you like but I used `python-dotenv`. Just create a `.env` file with your `OPENAI_API_KEY` then load it.

In [3]:
load_dotenv()

# API configuration
openai.api_key = os.getenv("OPENAI_API_KEY")

# for LangChain
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")
os.environ["SERPAPI_API_KEY"] = os.getenv("SERPAPI_API_KEY")

TypeError: str expected, not NoneType

In [4]:
def set_open_params(
    model="text-davinci-003",
    temperature=0.7,
    max_tokens=256,
    top_p=1,
    frequency_penalty=0,
    presence_penalty=0,
):
    """ set openai parameters"""

    openai_params = {}    

    openai_params['model'] = model
    openai_params['temperature'] = temperature
    openai_params['max_tokens'] = max_tokens
    openai_params['top_p'] = top_p
    openai_params['frequency_penalty'] = frequency_penalty
    openai_params['presence_penalty'] = presence_penalty
    return openai_params

def get_completion(params, prompt):
    """ GET completion from openai api"""

    response = openai.Completion.create(
        engine = params['model'],
        prompt = prompt,
        temperature = params['temperature'],
        max_tokens = params['max_tokens'],
        top_p = params['top_p'],
        frequency_penalty = params['frequency_penalty'],
        presence_penalty = params['presence_penalty'],
    )
    return response

Basic prompt example:

In [5]:
# basic example
params = set_open_params()

prompt = "The sky is"

response = get_completion(params, prompt)

In [6]:
response.choices[0].text

' clear\n\nThe sky is clear and blue with very few clouds. It is a beautiful day outside and the sun is shining brightly. The air is fresh and the breeze is gentle.'

In [7]:
IPython.display.Markdown(response.choices[0].text)

 clear

The sky is clear and blue with very few clouds. It is a beautiful day outside and the sun is shining brightly. The air is fresh and the breeze is gentle.

Try with different temperature to compare results:

In [8]:
params = set_open_params(temperature=0)
prompt = "The sky is"
response = get_completion(params, prompt)
IPython.display.Markdown(response.choices[0].text)

 blue

The sky is blue because of the way the atmosphere scatters sunlight. When sunlight passes through the atmosphere, the blue wavelengths are scattered more than the other colors, making the sky appear blue.

### 1.1 Text Summarization

In [9]:
params = set_open_params(temperature=0.7)
prompt = """Antibiotics are a type of medication used to treat bacterial infections. They work by either killing the bacteria or preventing them from reproducing, allowing the body's immune system to fight off the infection. Antibiotics are usually taken orally in the form of pills, capsules, or liquid solutions, or sometimes administered intravenously. They are not effective against viral infections, and using them inappropriately can lead to antibiotic resistance. 

Explain the above in one sentence:"""

response = get_completion(params, prompt)
IPython.display.Markdown(response.choices[0].text)


Antibiotics are medications used to treat bacterial infections by either killing or preventing the bacteria from reproducing, but they are not effective against viral infections and can lead to antibiotic resistance if misused.

Exercise: Instruct the model to explain the paragraph in one sentence like "I am 5". Do you see any differences?

### 1.2 Question Answering

In [10]:
prompt = """Answer the question based on the context below. Keep the answer short and concise. Respond "Unsure about answer" if not sure about the answer.

Context: Teplizumab traces its roots to a New Jersey drug company called Ortho Pharmaceutical. There, scientists generated an early version of the antibody, dubbed OKT3. Originally sourced from mice, the molecule was able to bind to the surface of T cells and limit their cell-killing potential. In 1986, it was approved to help prevent organ rejection after kidney transplants, making it the first therapeutic antibody allowed for human use.

Question: What was OKT3 originally sourced from?

Answer:"""

response = get_completion(params, prompt)
IPython.display.Markdown(response.choices[0].text)


 Mice

Context obtained from here: https://www.nature.com/articles/d41586-023-00400-x

Exercise: Edit prompt and get the model to respond that it isn't sure about the answer. 

### 1.3 Text Classification

In [11]:
prompt = """Classify the text into neutral, negative or positive.

Text: I think the food was okay.

Sentiment:"""

response = get_completion(params, prompt)
IPython.display.Markdown(response.choices[0].text)

 Neutral

Exercise: Modify the prompt to instruct the model to provide an explanation to the answer selected. 

### 1.4 Role Playing

In [12]:
prompt = """The following is a conversation with an AI research assistant. The assistant tone is technical and scientific.

Human: Hello, who are you?
AI: Greeting! I am an AI research assistant. How can I help you today?
Human: Can you tell me about the creation of blackholes?
AI:"""

response = get_completion(params, prompt)
IPython.display.Markdown(response.choices[0].text)

 Absolutely! Black holes are regions of spacetime where the gravitational force is so strong that nothing—not even light—can escape from it. They are formed when a massive star dies, collapsing under its own gravity and creating a point of infinite density known as a singularity.

In [16]:
prompt = """The following is a conversation with an AI research assistant. The assistant tone is consice and short, and trying to explain to a 5th grader.

Human: Hello, who are you?
AI: Greeting! I am an AI research assistant. How can I help you today?
Human: Can you tell me about the creation of blackholes?
AI:"""

response = get_completion(params, prompt)
IPython.display.Markdown(response.choices[0].text)

 Sure! Black holes form when a large star dies and its core collapses in on itself. This causes a huge amount of gravity that pulls in all the matter around it.

Exercise: Modify the prompt to instruct the model to keep AI responses concise and short.

### 1.5 Code Generation

In [13]:
prompt = "\"\"\"\nTable departments, columns = [DepartmentId, DepartmentName]\nTable students, columns = [DepartmentId, StudentId, StudentName]\nCreate a MySQL query for all students in the Computer Science Department\n\"\"\""

response = get_completion(params, prompt)
IPython.display.Markdown(response.choices[0].text)



SELECT StudentId, StudentName 
FROM students 
WHERE DepartmentId IN (SELECT DepartmentId 
                       FROM departments 
                       WHERE DepartmentName='Computer Science');

### 1.6 Reasoning

In [14]:
prompt = """The odd numbers in this group add up to an even number: 15, 32, 5, 13, 82, 7, 1. 

Solve by breaking the problem into steps. First, identify the odd numbers, add them, and indicate whether the result is odd or even."""

response = get_completion(params, prompt)
IPython.display.Markdown(response.choices[0].text)



Odd numbers: 15, 5, 13, 7, 1
Sum of odd numbers: 41
41 is an odd number.

Exercise: Improve the prompt to have a better structure and output format.

## 2. Advanced Prompting Techniques

Objectives:

- Cover more advanced techniques for prompting: few-shot, chain-of-thoughts,...

### 2.2 Few-shot prompts

In [17]:
prompt = """The odd numbers in this group add up to an even number: 4, 8, 9, 15, 12, 2, 1.
A: The answer is False.

The odd numbers in this group add up to an even number: 17,  10, 19, 4, 8, 12, 24.
A: The answer is True.

The odd numbers in this group add up to an even number: 16,  11, 14, 4, 8, 13, 24.
A: The answer is True.

The odd numbers in this group add up to an even number: 17,  9, 10, 12, 13, 4, 2.
A: The answer is False.

The odd numbers in this group add up to an even number: 15, 32, 5, 13, 82, 7, 1. 
A:"""

response = get_completion(params, prompt)
IPython.display.Markdown(response.choices[0].text)

 The answer is True.

### 2.3 Chain-of-Thought (CoT) Prompting

In [18]:
prompt = """The odd numbers in this group add up to an even number: 4, 8, 9, 15, 12, 2, 1.
A: Adding all the odd numbers (9, 15, 1) gives 25. The answer is False.

The odd numbers in this group add up to an even number: 15, 32, 5, 13, 82, 7, 1. 
A:"""

response = get_completion(params, prompt)
IPython.display.Markdown(response.choices[0].text)

 Adding all the odd numbers (15, 5, 13, 7, 1) gives 41. The answer is False.

### 2.4 Zero-shot CoT

In [19]:
prompt = """I went to the market and bought 10 apples. I gave 2 apples to the neighbor and 2 to the repairman. I then went and bought 5 more apples and ate 1. How many apples did I remain with?

Let's think step by step."""

response = get_completion(params, prompt)
IPython.display.Markdown(response.choices[0].text)



Initially, I had 10 apples.
I gave away 2 apples to the neighbor and 2 to the repairman.
So, I am left with 10 - (2 + 2) = 6 apples.
I bought 5 more apples, so now I have 6 + 5 = 11 apples.
Finally, I ate 1 apple, so I am left with 11 - 1 = 10 apples.

Therefore, I remained with 10 apples.

### 2.5 Self-Consistency
As an exercise, check examples in our [guide](https://github.com/dair-ai/Prompt-Engineering-Guide/blob/main/guides/prompts-advanced-usage.md#self-consistency) and try them here. 

### 2.6 Generate Knowledge Prompting

As an exercise, check examples in our [guide](https://github.com/dair-ai/Prompt-Engineering-Guide/blob/main/guides/prompts-advanced-usage.md#generated-knowledge-prompting) and try them here. 

### 2.6 PAL - Code as Reasoning

We are developing a simple application that's able to reason about the question being asked through code. 

Specifically, the application takes in some data and answers a question about the data input. The prompt includes a few exemplars which are adopted from [here](https://github.com/reasoning-machines/pal/blob/main/pal/prompt/penguin_prompt.py).  

In [20]:
# lm instance
llm = OpenAI(model_name='text-davinci-003', temperature=0)

In [21]:
question = "Which is the oldest penguin?"

In [22]:
PENGUIN_PROMPT = '''
"""
Q: Here is a table where the first line is a header and each subsequent line is a penguin:
name, age, height (cm), weight (kg) 
Louis, 7, 50, 11
Bernard, 5, 80, 13
Vincent, 9, 60, 11
Gwen, 8, 70, 15
For example: the age of Louis is 7, the weight of Gwen is 15 kg, the height of Bernard is 80 cm. 
We now add a penguin to the table:
James, 12, 90, 12
How many penguins are less than 8 years old?
"""
# Put the penguins into a list.
penguins = []
penguins.append(('Louis', 7, 50, 11))
penguins.append(('Bernard', 5, 80, 13))
penguins.append(('Vincent', 9, 60, 11))
penguins.append(('Gwen', 8, 70, 15))
# Add penguin James.
penguins.append(('James', 12, 90, 12))
# Find penguins under 8 years old.
penguins_under_8_years_old = [penguin for penguin in penguins if penguin[1] < 8]
# Count number of penguins under 8.
num_penguin_under_8 = len(penguins_under_8_years_old)
answer = num_penguin_under_8
"""
Q: Here is a table where the first line is a header and each subsequent line is a penguin:
name, age, height (cm), weight (kg) 
Louis, 7, 50, 11
Bernard, 5, 80, 13
Vincent, 9, 60, 11
Gwen, 8, 70, 15
For example: the age of Louis is 7, the weight of Gwen is 15 kg, the height of Bernard is 80 cm.
Which is the youngest penguin?
"""
# Put the penguins into a list.
penguins = []
penguins.append(('Louis', 7, 50, 11))
penguins.append(('Bernard', 5, 80, 13))
penguins.append(('Vincent', 9, 60, 11))
penguins.append(('Gwen', 8, 70, 15))
# Sort the penguins by age.
penguins = sorted(penguins, key=lambda x: x[1])
# Get the youngest penguin's name.
youngest_penguin_name = penguins[0][0]
answer = youngest_penguin_name
"""
Q: Here is a table where the first line is a header and each subsequent line is a penguin:
name, age, height (cm), weight (kg) 
Louis, 7, 50, 11
Bernard, 5, 80, 13
Vincent, 9, 60, 11
Gwen, 8, 70, 15
For example: the age of Louis is 7, the weight of Gwen is 15 kg, the height of Bernard is 80 cm.
What is the name of the second penguin sorted by alphabetic order?
"""
# Put the penguins into a list.
penguins = []
penguins.append(('Louis', 7, 50, 11))
penguins.append(('Bernard', 5, 80, 13))
penguins.append(('Vincent', 9, 60, 11))
penguins.append(('Gwen', 8, 70, 15))
# Sort penguins by alphabetic order.
penguins_alphabetic = sorted(penguins, key=lambda x: x[0])
# Get the second penguin sorted by alphabetic order.
second_penguin_name = penguins_alphabetic[1][0]
answer = second_penguin_name
"""
{question}
"""
'''.strip() + '\n'

Now that we have the prompt and question. We can send it to the model. It should output the steps, in code, needed to get the solution to the answer.

In [23]:
llm_out = llm(PENGUIN_PROMPT.format(question=question))
print(llm_out)

# Put the penguins into a list.
penguins = []
penguins.append(('Louis', 7, 50, 11))
penguins.append(('Bernard', 5, 80, 13))
penguins.append(('Vincent', 9, 60, 11))
penguins.append(('Gwen', 8, 70, 15))
# Sort the penguins by age.
penguins = sorted(penguins, key=lambda x: x[1], reverse=True)
# Get the oldest penguin's name.
oldest_penguin_name = penguins[0][0]
answer = oldest_penguin_name


In [24]:
exec(llm_out)
print(answer)

Vincent


That's the correct answer! Vincent is the oldest penguin. 

Exercise: Try a different question and see what's the result.

---

# 3. Tools and Applications

Objective:

- Demonstrate how to use LangChain to demonstrate simple applications using prompting techniques and LLMs

### 3.1 LLMs & External Tools

Example adopted from the [LangChain documentation](https://langchain.readthedocs.io/en/latest/modules/agents/getting_started.html).

In [25]:
from langchain.agents import load_tools
from langchain.agents import initialize_agent

In [26]:
llm = OpenAI(temperature=0)

tools = load_tools(["serpapi", "llm-math"], llm=llm)
agent = initialize_agent(tools, llm, agent="zero-shot-react-description", verbose=True)

ValidationError: 1 validation error for SerpAPIWrapper
__root__
  Did not find serpapi_api_key, please add an environment variable `SERPAPI_API_KEY` which contains it, or pass  `serpapi_api_key` as a named parameter. (type=value_error)

In [24]:
# run the agent
agent.run("Who is Olivia Wilde's boyfriend? What is his current age raised to the 0.23 power?")



> Entering new AgentExecutor chain...
 I need to find out who Olivia Wilde's boyfriend is and then calculate his age raised to the 0.23 power.
Action: Search
Action Input: "Olivia Wilde boyfriend"
Observation: Jason Sudeikis
Thought: I need to find out Jason Sudeikis' age
Action: Search
Action Input: "Jason Sudeikis age"
Observation: 47 years
Thought: I need to calculate 47 raised to the 0.23 power
Action: Calculator
Action Input: 47^0.23
Observation: Answer: 2.4242784855673896

Thought: I now know the final answer
Final Answer: Jason Sudeikis, Olivia Wilde's boyfriend, is 47 years old and his age raised to the 0.23 power is 2.4242784855673896.

> Finished chain.


"Jason Sudeikis, Olivia Wilde's boyfriend, is 47 years old and his age raised to the 0.23 power is 2.4242784855673896."

### 3.2 Data-Augmented Generation

First, we need to download the data we want to use as source to augment generation.

Code example adopted from [LangChain Documentation](https://langchain.readthedocs.io/en/latest/modules/chains/combine_docs_examples/qa_with_sources.html). We are only using the examples for educational purposes.

Prepare the data first:

In [25]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.embeddings.cohere import CohereEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores.elastic_vector_search import ElasticVectorSearch
from langchain.vectorstores import Chroma
from langchain.docstore.document import Document
from langchain.prompts import PromptTemplate

In [26]:
with open('./state_of_the_union.txt') as f:
    state_of_the_union = f.read()
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_text(state_of_the_union)

embeddings = OpenAIEmbeddings()

In [27]:
docsearch = Chroma.from_texts(texts, embeddings, metadatas=[{"source": str(i)} for i in range(len(texts))])

Running Chroma using direct local API.
Using DuckDB in-memory for database. Data will be transient.


In [28]:
query = "What did the president say about Justice Breyer"
docs = docsearch.similarity_search(query)

Let's quickly test it:

In [29]:
from langchain.chains.qa_with_sources import load_qa_with_sources_chain
from langchain.llms import OpenAI

In [30]:
chain = load_qa_with_sources_chain(OpenAI(temperature=0), chain_type="stuff")
query = "What did the president say about Justice Breyer"
chain({"input_documents": docs, "question": query}, return_only_outputs=True)

{'output_text': ' The president thanked Justice Breyer for his service.\nSOURCES: 30-pl'}

Let's try a question with a custom prompt:

In [31]:
template = """Given the following extracted parts of a long document and a question, create a final answer with references ("SOURCES"). 
If you don't know the answer, just say that you don't know. Don't try to make up an answer.
ALWAYS return a "SOURCES" part in your answer.
Respond in Spanish.

QUESTION: {question}
=========
{summaries}
=========
FINAL ANSWER IN SPANISH:"""

# create a prompt template
PROMPT = PromptTemplate(template=template, input_variables=["summaries", "question"])

# query 
chain = load_qa_with_sources_chain(OpenAI(temperature=0), chain_type="stuff", prompt=PROMPT)
query = "What did the president say about Justice Breyer?"
chain({"input_documents": docs, "question": query}, return_only_outputs=True)

{'output_text': '\nEl Presidente no dijo nada acerca de la Justicia Breyer.\n\nFUENTES: 30, 31, 33'}

Exercise: Try using a different dataset from the internet and try different prompt, including all the techniques you learned in the lecture.